# Random Forests

Les arbres de décision nous demandent de prendre une décision difficile : un arbre profond, avec de nombreuses feuilles, sera trop précis car chaque prédiction n'aura été calculée que sur un nombre restreint de maisons. Mais un arbre peu profond avec peu de feuilles sera peu performant car il ne parvient pas à saisir l'importance des différentes caractéristiques de notre jeu de données.

Même les techniques de modélisation les plus sophistiquées d'aujourd'hui sont confrontées à cette tension entre sous-adaptation et sur-adaptation. Cependant, de nombreux modèles présentent des idées astucieuses qui peuvent conduire à de meilleures performances, c'est le cas de l'algorithme de *random forest* (forêt aléatoire).

Une *random forest* génère de nombreux arbres de décision différents et effectue une prédiction en faisant la moyenne des prédictions de chaque arbre. Ses prédictions sont généralement bien meilleures que celles d'un arbre de décision unique et les paramètres par défaut sont souvent suffisant.

Son désavantage : l'algorithme agit comme une boîte noire, et il est difficile d'expliquer ses choix.

Cet algorithme revêt différentes formes. Il peut être utilisé pour prédire une valeur discrète (*regressor*) ou bien prédire une classe (*classifier*).

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from pprint import pprint

df = pd.read_csv("data/iowa_housing.csv")

In [ ]:
y = df['saleprice']

feature_names = ['lotarea',
                 'yearbuilt',
                 '1stflrsf',
                 '2ndflrsf',
                 'fullbath',
                 'bedroomabvgr',
                 'totrmsabvgrd',]

X = df[feature_names]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

iowa_rf_model = RandomForestRegressor(random_state=42)
iowa_rf_model.fit(X_train, y_train)
y_pred = iowa_rf_model.predict(X_test)
mean_absolute_error(y_test, y_pred)

In [ ]:
# le R², coefficient de détermination.
# proportion de la variance d’une variable dépendante (y) 
# qui s’explique par une ou plusieurs variables indépendantes (X) dans le modèle de régression

iowa_rf_model.score(X_test, y_test)

## Optimisation des paramètres

In [ ]:
# Affichons les paramètres de notre modèle

iowa_rf_model.get_params()

### Random Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
# Number of features to consider at every split
max_features = [1, 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]#.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,}

pprint(random_grid)

In [ ]:
# On calcule les différentes possibilités.
possibilities = 1
for param_list in random_grid.values():
    possibilities *= len(param_list)
possibilities

In [ ]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = iowa_rf_model,
                               param_distributions = random_grid,
                               n_iter=10,
                               cv=3,
                               verbose=2,
                               random_state=42,
                               n_jobs=-1)
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
# La meilleure des random forest

rf_random.best_score_

In [ ]:
# Ses paramètres

rf_random.best_params_

In [ ]:
# Son score

rf_random.best_estimator_.score(X_test, y_test)

### *Fine Tuning*

In [ ]:
# Pour rappel voici les meilleurs paramètres :

{'n_estimators': 1400,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 30,
 'bootstrap': True}

In [ ]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [20, 30, 40],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [3, 5, 7],
    'n_estimators': [1200, 1400, 1600]
}

# Instantiate the grid search model
grid_search_rf = GridSearchCV(estimator=iowa_rf_model,
                              param_grid=param_grid, 
                              cv=2,
                              n_jobs=-1,
                              verbose=1)

In [ ]:
grid_search_rf.fit(X_train, y_train)

In [ ]:
grid_search_rf.best_params_

In [ ]:
grid_search_rf.score(X_test, y_test)

In [ ]:
grid_search_rf.best_estimator_.score(X_test, y_test)

### Modèle final

In [ ]:
final_rf = grid_search_rf.best_estimator_

from joblib import dump, load
dump(final_rf, 'best_iowa_rf_model.joblib') 